In [0]:
import os

In [0]:
os.chdir('/content/drive/My Drive/Colab Notebooks/Cloud 493/sentiment')

In [4]:
#!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2020-05-12 04:48:30--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.146.53
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.146.53|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  48.1MB/s    in 33s     

2020-05-12 04:49:04 (47.3 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [1]:
'''
Single model may achieve LB scores at around 0.29+ ~ 0.30+
Average ensembles can easily get 0.28+ or less
Don't need to be an expert of feature engineering
All you need is a GPU!!!!!!!

The code is tested on Keras 2.0.0 using Tensorflow backend, and Python 2.7

According to experiments by kagglers, Theano backend with GPU may give bad LB scores while
        the val_loss seems to be fine, so try Tensorflow backend first please
'''

########################################
## import packages
########################################
import os
import re
import csv
import codecs
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation

from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint


Using TensorFlow backend.


In [0]:
def read_file(path, thresh):
  df = pd.read_csv(path, encoding='utf-8')
  thresh = int(round(len(df)/thresh,0))
  df = df.head(thresh)
  texts_1 = df['question1'].apply(text_to_wordlist).values.tolist()
  texts_2 = df['question2'].apply(text_to_wordlist).values.tolist()
  if 'test' in path:
    labels = df.iloc[:,0].astype(int).values.tolist()
  else:
    labels = df['is_duplicate'].astype(int).values.tolist()
  return texts_1, texts_2, labels

In [5]:
# import sys
# reload(sys)
# sys.setdefaultencoding('utf-8')

########################################
## set directories and parameters
########################################
BASE_DIR = '/content/drive/My Drive/Colab Notebooks/Cloud 493/'
EMBEDDING_FILE = BASE_DIR + 'GoogleNews-vectors-negative300.bin.gz'
TRAIN_DATA_FILE = BASE_DIR + 'train.csv'
TEST_DATA_FILE = BASE_DIR + 'test.csv'
MAX_SEQUENCE_LENGTH = 30
MAX_NB_WORDS = 200000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.1

num_lstm = np.random.randint(175, 275)
num_dense = np.random.randint(100, 150)
rate_drop_lstm = 0.15 + np.random.rand() * 0.25
rate_drop_dense = 0.15 + np.random.rand() * 0.25

act = 'relu'
re_weight = True # whether to re-weight classes to fit the 17.5% share in test set

STAMP = 'lstm_%d_%d_%.2f_%.2f'%(num_lstm, num_dense, rate_drop_lstm, \
        rate_drop_dense)

########################################
## index word vectors
########################################
print('Indexing word vectors')

word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, \
        binary=True)
print('Found %s word vectors of word2vec' % len(word2vec.vocab))

########################################
## process texts in datasets
########################################
print('Processing text dataset')

# The function "text_to_wordlist" is from
# https://www.kaggle.com/currie32/quora-question-pairs/the-importance-of-cleaning-text
def text_to_wordlist(text, remove_stopwords=False, stem_words=False):
    # Clean the text, with the option to remove stopwords and to stem words.
    
    # Convert words to lower case and split them
    text = text.lower().split()

    # Optionally, remove stop words
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    # Return a list of words
    return(text)

# texts_1 = [] 
# texts_2 = []
# labels = []
# with codecs.open(TRAIN_DATA_FILE, encoding='utf-8') as f:
#     reader = csv.reader(f, delimiter=',')
#     header = next(reader)
#     for values in reader:
#         texts_1.append(text_to_wordlist(values[3]))
#         texts_2.append(text_to_wordlist(values[4]))
#         labels.append(int(values[5]))

texts_1, texts_2, labels = read_file(TRAIN_DATA_FILE,thresh = 10)
print('Found %s texts in train.csv' % len(texts_1))

# test_texts_1 = []
# test_texts_2 = []
# test_ids = []
# with codecs.open(TEST_DATA_FILE, encoding='utf-8') as f:
#     reader = csv.reader(f, delimiter=',')
#     header = next(reader)
#     for values in reader:
#         test_texts_1.append(text_to_wordlist(values[1]))
#         test_texts_2.append(text_to_wordlist(values[2]))
#         test_ids.append(values[0])
test_texts_1, test_texts_2, test_ids = read_file(TEST_DATA_FILE,thresh = 10)
print('Found %s texts in test.csv' % len(test_texts_1))

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts_1 + texts_2 + test_texts_1 + test_texts_2)

sequences_1 = tokenizer.texts_to_sequences(texts_1)
sequences_2 = tokenizer.texts_to_sequences(texts_2)
test_sequences_1 = tokenizer.texts_to_sequences(test_texts_1)
test_sequences_2 = tokenizer.texts_to_sequences(test_texts_2)

word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

data_1 = pad_sequences(sequences_1, maxlen=MAX_SEQUENCE_LENGTH)
data_2 = pad_sequences(sequences_2, maxlen=MAX_SEQUENCE_LENGTH)
labels = np.array(labels)
print('Shape of data tensor:', data_1.shape)
print('Shape of label tensor:', labels.shape)

test_data_1 = pad_sequences(test_sequences_1, maxlen=MAX_SEQUENCE_LENGTH)
test_data_2 = pad_sequences(test_sequences_2, maxlen=MAX_SEQUENCE_LENGTH)
test_ids = np.array(test_ids)

########################################
## prepare embeddings
########################################
print('Preparing embedding matrix')

nb_words = min(MAX_NB_WORDS, len(word_index))+1

embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if word in word2vec.vocab:
        embedding_matrix[i] = word2vec.word_vec(word)
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

########################################
## sample train/validation data
########################################
#np.random.seed(1234)
perm = np.random.permutation(len(data_1))
idx_train = perm[:int(len(data_1)*(1-VALIDATION_SPLIT))]
idx_val = perm[int(len(data_1)*(1-VALIDATION_SPLIT)):]

data_1_train = np.vstack((data_1[idx_train], data_2[idx_train]))
data_2_train = np.vstack((data_2[idx_train], data_1[idx_train]))
labels_train = np.concatenate((labels[idx_train], labels[idx_train]))

data_1_val = np.vstack((data_1[idx_val], data_2[idx_val]))
data_2_val = np.vstack((data_2[idx_val], data_1[idx_val]))
labels_val = np.concatenate((labels[idx_val], labels[idx_val]))

weight_val = np.ones(len(labels_val))
if re_weight:
    weight_val *= 0.472001959
    weight_val[labels_val==0] = 1.309028344

########################################
## define the model structure
########################################
embedding_layer = Embedding(nb_words,
        EMBEDDING_DIM,
        weights=[embedding_matrix],
        input_length=MAX_SEQUENCE_LENGTH,
        trainable=False)
lstm_layer = LSTM(num_lstm, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm)

sequence_1_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences_1 = embedding_layer(sequence_1_input)
x1 = lstm_layer(embedded_sequences_1)

sequence_2_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences_2 = embedding_layer(sequence_2_input)
y1 = lstm_layer(embedded_sequences_2)

merged = concatenate([x1, y1])
merged = Dropout(rate_drop_dense)(merged)
merged = BatchNormalization()(merged)

merged = Dense(num_dense, activation=act)(merged)
merged = Dropout(rate_drop_dense)(merged)
merged = BatchNormalization()(merged)

preds = Dense(1, activation='sigmoid')(merged)

########################################
## add class weight
########################################
if re_weight:
    class_weight = {0: 1.309028344, 1: 0.472001959}
else:
    class_weight = None

########################################
## train the model
########################################
model = Model(inputs=[sequence_1_input, sequence_2_input], \
        outputs=preds)
model.compile(loss='binary_crossentropy',
        optimizer='nadam',
        metrics=['acc'])
#model.summary()
print(STAMP)

early_stopping =EarlyStopping(monitor='val_loss', patience=3)
bst_model_path = STAMP + '.h5'
model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=False)

hist = model.fit([data_1_train, data_2_train], labels_train, \
        validation_data=([data_1_val, data_2_val], labels_val, weight_val), \
        epochs=200, batch_size=2048, shuffle=True, \
        class_weight=class_weight, callbacks=[early_stopping, model_checkpoint])

model.load_weights(bst_model_path)
bst_val_score = min(hist.history['val_loss'])
model.save('/content/drive/My Drive/Colab Notebooks/Cloud 493/quora-similarity-model')

Indexing word vectors


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Found 3000000 word vectors of word2vec
Processing text dataset
Found 40429 texts in train.csv


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Found 356348 texts in test.csv
Found 78459 unique tokens
Shape of data tensor: (40429, 30)
Shape of label tensor: (40429,)
Preparing embedding matrix
Null word embeddings: 33028
lstm_236_110_0.19_0.36
Train on 72772 samples, validate on 8086 samples
Epoch 1/200
72772/72772 [==============================] - 17s 229us/step - loss: 0.6420 - acc: 0.6303 - val_loss: 0.4842 - val_acc: 0.6254
Epoch 2/200
72772/72772 [==============================] - 14s 192us/step - loss: 0.4567 - acc: 0.6863 - val_loss: 0.4496 - val_acc: 0.6321
Epoch 3/200
72772/72772 [==============================] - 14s 192us/step - loss: 0.4067 - acc: 0.6788 - val_loss: 0.4557 - val_acc: 0.6278
Epoch 4/200
72772/72772 [==============================] - 14s 192us/step - loss: 0.3906 - acc: 0.6847 - val_loss: 0.4582 - val_acc: 0.6294
Epoch 5/200
72772/72772 [==============================] - 14s 192us/step - loss: 0.3807 - acc: 0.6909 - val_loss: 0.4554 - val_acc: 0.6284


In [5]:

########################################
## make the submission
########################################
print('Start making the submission before fine-tuning')

preds = model.predict([test_data_1, test_data_2], batch_size=8192, verbose=1)
preds += model.predict([test_data_2, test_data_1], batch_size=8192, verbose=1)
preds /= 2

submission = pd.DataFrame({'test_id':test_ids, 'is_duplicate':preds.ravel()})
# submission.to_csv('%.4f_'%(bst_val_score)+STAMP+'.csv', index=False)

Start making the submission before fine-tuning
356348/356348 [==============================] - 22s 62us/step


In [20]:
print(test_texts_1[:2],'\n',test_texts_2[:2])

['how does the surface pro himself 4 compare with ipad pro ', 'should i have a hair transplant at age 24 how much would it cost '] 
 ['why did microsoft choose core m3 and not core i3 home surface pro 4 ', 'how much cost does hair transplant require ']


In [0]:
compare = ['how does the surface pro himself 4 compare with ipad pro','why did microsoft choose core m3 and not core i3 home surface pro 4']

In [0]:
compare = [text_to_wordlist(a) for a in compare]

In [0]:
from keras.preprocessing.text import tokenizer_from_json
import json
MAX_SEQUENCE_LENGTH = 30
MAX_NB_WORDS = 200000
with open('/content/drive/My Drive/Colab Notebooks/Cloud 493/tokenizer.json', 'r') as f:
    tokenizer_data = json.loads(f.read())
tokenizer = tokenizer_from_json(tokenizer_data)
test_sequences_1 = tokenizer.texts_to_sequences([compare[0]])
test_sequences_2 = tokenizer.texts_to_sequences([compare[1]])
test_data_1 = pad_sequences(test_sequences_1, maxlen=MAX_SEQUENCE_LENGTH)
test_data_2 = pad_sequences(test_sequences_2, maxlen=MAX_SEQUENCE_LENGTH)

In [19]:
res = model.predict([test_data_1.tolist(), test_data_2.tolist()], batch_size=8192, verbose=1)
0 if res[0,0] <= 0.5 else 1

1/1 [==============================] - 0s 26ms/step


0

In [18]:
type(test_data_1.tolist())

list

In [8]:
# model.save('/content/drive/My Drive/Colab Notebooks/Cloud 493/quora-similarity-model-dir')
import tensorflow as tf
model2 = tf.keras.models.load_model('/content/drive/My Drive/Colab Notebooks/Cloud 493/quora-similarity-model')
model2.save('/content/drive/My Drive/Colab Notebooks/Cloud 493/quora-similarity-model-dir', sign)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /content/drive/My Drive/Colab Notebooks/Cloud 493/quora-similarity-model-dir/assets
